In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans

In [2]:
from glob import glob

In [3]:
import json

In [16]:
# list of headers
list(df.columns.values)

['address',
 'attributes_Alcohol',
 'attributes_Ambience',
 'attributes_BusinessAcceptsCreditCards',
 'attributes_GoodForKids',
 'attributes_GoodForMeal',
 'attributes_HasTV',
 'attributes_NoiseLevel',
 'attributes_OutdoorSeating',
 'attributes_RestaurantsAttire',
 'attributes_RestaurantsGoodForGroups',
 'attributes_RestaurantsPriceRange2',
 'attributes_RestaurantsReservations',
 'attributes_RestaurantsTableService',
 'attributes_RestaurantsTakeOut',
 'attributes_WiFi',
 'business_id',
 'categories',
 'city',
 'name',
 'review_count',
 'stars']

In [17]:
unwanted_columns = ['attributes_AcceptsInsurance','attributes_AgesAllowed','attributes_BusinessAcceptsBitcoin','attributes_DietaryRestrictions','attributes_Open24Hours',
                    'attributes_BusinessParking','attributes_ByAppointmentOnly','attributes_Corkage','attributes_DogsAllowed','attributes_DriveThru','attributes_GoodForDancing',
                    'attributes_Caters','attributes_CoatCheck','attributes_Corkage','attributes_HairSpecializesIn','attributes_RestaurantsDelivery','attributes_Smoking',
                    'attributes_Music','attributes_RestaurantsCounterService','attributes_BikeParking',
                    'attributes_WheelchairAccessible','hours_Friday','hours_Monday','hours_Saturday','hours_Sunday','hours_Thursday','hours_Tuesday','hours_Wednesday',
                    'is_open','attributes_BestNights','attributes_HappyHour','attributes_BYOB','attributes_BYOBCorkage',
                    'latitude', 'longitude','neighborhood','postal_code','state']

# Convert Json to CSV file 
# 1) only select restaurants by filtering categories
# 2) select top 5000 restaurants with highest number of reviews

In [18]:
def convert(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob

for json_filename in glob('business.json'):
    csv_filename = '%s.csv' % json_filename[:-2]
    print 'Converting %s to %s' % (json_filename, csv_filename)
    df = pd.DataFrame([convert(line) for line in file(json_filename)])
    df = df[df.categories.apply(lambda x: 'Restaurants' in x)]
    df = df.sort('review_count', ascending=False)
    df = df.drop(unwanted_columns, axis=1)
    df = df.head(5000)
    df.to_csv(csv_filename, encoding='utf-8', index=False)

Converting business.json to business.js.csv


/Users/lanniemiao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [21]:
df

,address,attributes_Alcohol,attributes_Ambience,attributes_BusinessAcceptsCreditCards,attributes_GoodForKids,attributes_GoodForMeal,attributes_HasTV,attributes_NoiseLevel,attributes_OutdoorSeating,attributes_RestaurantsAttire,...,attributes_RestaurantsReservations,attributes_RestaurantsTableService,attributes_RestaurantsTakeOut,attributes_WiFi,business_id,categories,city,name,review_count,stars
103325,3655 Las Vegas Blvd S,full_bar,"{u'romantic': True, u'intimate': False, u'clas...",True,True,"{u'dessert': False, u'latenight': False, u'lun...",False,average,True,casual,...,True,True,False,no,4JNXUYY8wbaaDmk3BPzlWw,"French,Steakhouses,Breakfast & Brunch,Restaurants",Las Vegas,Mon Ami Gabi,6979,4.0
125404,3570 S Las Vegas Blvd,full_bar,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': False, u'latenight': False, u'lun...",False,average,False,casual,...,False,True,False,no,RESDUcs7fIiihp38-d6_6g,"Food,Restaurants,Buffets,Breakfast & Brunch,Sa...",Las Vegas,Bacchanal Buffet,6417,4.0
45588,3708 Las Vegas Blvd S,full_bar,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': False, u'latenight': False, u'lun...",False,average,False,casual,...,False,True,False,no,K7lWdNUhCbcnEvI0NhGewg,"Buffets,Breakfast & Brunch,Restaurants",Las Vegas,Wicked Spoon,5632,3.5
114018,3667 Las Vegas Blvd S,full_bar,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': False, u'latenight': False, u'lun...",True,loud,False,casual,...,False,True,True,no,cYwJA2A6I12KNkm2rtXd5g,"American (Traditional),Restaurants,Burgers",Las Vegas,Gordon Ramsay BurGR,5429,4.0
100916,3667 Las Vegas Blvd S,beer_and_wine,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': False, u'latenight': True, u'lunc...",True,average,False,casual,...,False,False,True,free,DkYS3arLOhA8si5uUEmHOw,"Food Delivery Services,Restaurants,Caterers,Sa...",Las Vegas,Earl of Sandwich,4789,4.5
104769,3535 Las Vegas Blvd,full_bar,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': False, u'latenight': False, u'lun...",True,average,False,casual,...,False,True,True,no,f4x1YBxkLrZg652xt2KR5g,"American (New),Restaurants,Breakfast & Brunch",Las Vegas,Hash House A Go Go,4371,4.0
106981,"Caesars Palace Las Vegas Hotel And Casino, 357...",full_bar,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': True, u'latenight': False, u'lunc...",True,average,True,casual,...,True,True,True,no,eoHdUeQDNgQ6WYEnP2aiRw,"Social Clubs,Restaurants,Desserts,Food,America...",Las Vegas,Serendipity 3,3913,3.0
19669,3131 Las Vegas Blvd S,full_bar,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': False, u'latenight': False, u'lun...",False,average,False,casual,...,False,True,False,no,2weQS-RnoOBhb1KsHKyoSQ,"Buffets,Restaurants",Las Vegas,The Buffet,3873,3.5
92918,"953 E Sahara Ave, Ste A5",beer_and_wine,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': False, u'latenight': False, u'lun...",False,average,False,casual,...,True,True,True,no,KskYqH1Bi7Z_61pH6Om8pg,"Automotive,Car Dealers,Restaurants,Thai,Nightl...",Las Vegas,Lotus of Siam,3838,4.0
68904,3600 S Las Vegas Blvd,full_bar,"{u'romantic': False, u'intimate': False, u'cla...",True,True,"{u'dessert': False, u'latenight': False, u'lun...",False,average,False,casual,...,False,True,False,free,ujHiaprwCQ5ewziu0Vi9rw,"American (New),Buffets,Restaurants",Las Vegas,The Buffet at Bellagio,3700,3.5


In [83]:
for json_filename in glob('review.json'):
    csv_filename = '%s.csv' % json_filename[:-2]
    print 'Converting %s to %s' % (json_filename, csv_filename)
    review_df = pd.DataFrame([convert(line) for line in file(json_filename)])
    review_df = review_df[review_df.business_id.apply(lambda x: x in all_business)]
    review_df.to_csv(csv_filename, encoding='utf-8', index=False)

Converting review.json to review.js.csv


# Validation using power users

In [100]:
review_df

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
998,tulUhFYMvBkYHsjmn30A9w,1,2013-11-19,0,FsS5TUFPI8QJEE60-HR3dw,2,Wished it was better..\nAfter watching man vs....,1,bWh4k_cCuVt5GLVd33xIxg
999,tulUhFYMvBkYHsjmn30A9w,1,2014-12-18,0,7xGHiLP1vAaGmX6srC_XXw,4,"Decor and service leave much to be desired, bu...",0,nQ4e81UdfczimYcIUtO3HA
1000,tulUhFYMvBkYHsjmn30A9w,1,2014-09-12,0,ZWlXWc9LHPLiOksrp-enyw,5,My husband and I ate here tonight for the firs...,0,gJPa95ZRozMhiOqvENpspA
1001,tulUhFYMvBkYHsjmn30A9w,1,2012-02-28,1,KpRwKYyQ93ypyDSdA7IXfw,2,Don't believe the hype. Nooooo! \n\nIn the Cle...,5,bAwfPH4lXNzgcYp9JFy6ow
1002,tulUhFYMvBkYHsjmn30A9w,3,2014-10-06,6,OZvrgp4vWBsYqIt3-YMSEw,3,Don't believe the hype!\n\nAfter seeing this l...,10,BjtJ3VkMOxV2Lan037AFuw
1003,tulUhFYMvBkYHsjmn30A9w,0,2015-10-07,0,LDln5I1QXIIinI74RmjuSQ,1,Sauce spicy and blan absolutely no flavor the ...,0,ZfqrQVvCfnhlqma6bkEWJw
1004,tulUhFYMvBkYHsjmn30A9w,1,2016-06-24,2,_Ob9cAp8dT0w1X5GsZP6vg,5,Seti's truck wasn't around so we decided to he...,1,AiQL1INckKUk1Bx2WD8ozw
1005,tulUhFYMvBkYHsjmn30A9w,0,2015-04-14,0,g9VGCBuaZ079NtNNfmVPaA,1,"The location on 79th Carnegie, is a hot ghetto...",0,J50BanBFNY4naDYAJRAZAA
1006,tulUhFYMvBkYHsjmn30A9w,0,2011-04-19,1,3luqqY_D_XQCj_JUOaCL2w,2,Let me just start by saying that this place di...,0,mzexPXl2iuASVakpjODhrQ
1007,tulUhFYMvBkYHsjmn30A9w,1,2014-08-31,0,Tfkyj1e0-_D_t2uos9W5gg,4,"I love Hot Sauce Wiliams. Well, I love their f...",0,nNtI1xRXcZs5R3sBEa8xqw


In [102]:
review_df['user_id'].value_counts()

bLbSNkLggFnqwNNzzq-Ijw    685
d_TBs6J3twMy9GChqUEXkg    610
PKEzKWv_FktMm2mGPjwd0Q    553
cMEtAiW60I5wE_vLfTxoJQ    543
U4INQZOPSUaj8hMjLlZ3KA    481
UYcmGbelzRa0Q6JqzLoguw    462
DK57YibC5ShBmqQl97CKog    415
C2C0GPKvzWWnP57Os9eQ0w    399
ELcQDlf69kb-ihJfxZyL0A    387
tH0uKD-vNwMoEc3Xk3Cbdg    377
n86B7IkbU20AkxlFX_5aew    359
N3oNEwh0qgPqPP3Em6wJXw    357
kjeX2RXvW7RhBbD2QLd5jA    349
QJI9OSEn6ujRCtrX06vs1w    316
8DEyKVyplnOcSKx39vatbg    315
3nDUQBjKyVor5wV0reJChg    291
qewG3X2O4X6JKskxyyqFwQ    290
iDlkZO2iILS8Jwfdy7DP9A    285
S9Jw00eZHVj5_0sOM_C5Rg    281
dIIKEfOgo0KqUfGQvGikPg    278
PeLGa5vUR8_mcsn-fn42Jg    277
M9rRM6Eo5YbKLKMG5QiIPA    273
CxDOIDnH8gp9KXzpBHJYXw    273
V-BbqKqO8anwplGRx9Q5aQ    269
RBZ_kMjowV0t6_nv2UKaDQ    267
pwQ8E-hbDUJe3qtKt_IBBg    264
JaqcCU3nxReTW2cBLHounA    263
SCo1UBoeN3bhRMkSYuiX1A    261
3nIuSCZk5f_2WWYMLN7h3w    259
y3FcL4bLy0eLlkb0SDPnBQ    253
                         ... 
SvI95OpIPLC_8OizB51q4A      1
cPYAmPOyLaFGgS-xkjFcEw      1
7z0dGbuUAQ

In [140]:
user1_reviews_df = review_df.loc[review_df['user_id'] == 'cMEtAiW60I5wE_vLfTxoJQ']

In [141]:
user1_reviews_df = user1_reviews_df.sort('stars', ascending=False)

/Users/lanniemiao/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [142]:
user1_reviews_df

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
5913,lCd20nDS9rFHW_2KViqWUA,1,2015-07-04,1,SoF7qlkOh80Z9xblx6MDfw,5,This place is ridiculously good...so good I ca...,4,cMEtAiW60I5wE_vLfTxoJQ
1635994,QARy4UUOQogMjDp9jP-YVg,0,2013-01-30,0,MPFZt68Xgoi5vXY45fYs7w,5,I was really impressed with this place! The d...,0,cMEtAiW60I5wE_vLfTxoJQ
3866160,Tw3miGKZHtmxmaQZIYFRrA,0,2012-11-03,0,nmrXaPFSUpIlXSWERYYTpw,5,Great new addition to the Central/Camelback co...,0,cMEtAiW60I5wE_vLfTxoJQ
1793659,oj8qk3W7O9oq15VIs-vYJQ,2,2011-05-06,0,C5t4yBkSs6uy-_2AmqYEMQ,5,One of my favorite places in Scottsdale. This...,1,cMEtAiW60I5wE_vLfTxoJQ
3822671,Rg43a31sV7gNLZMjy-ZdCg,0,2013-11-17,0,yH1mM3iVGotl4iqUOLNDZw,5,Such a great place! We stopped by this aftern...,2,cMEtAiW60I5wE_vLfTxoJQ
3790567,Noi53T0PWNEN9mQRS3-Ncg,0,2015-11-28,0,K-xKU62rckdg3YbPZqsyMg,5,"I had always heard this place was amazing, but...",2,cMEtAiW60I5wE_vLfTxoJQ
1910362,WB7346WSdkTHvtHfWPilaA,5,2015-12-06,5,QszUgcqQ0lNb-5ytFnHW6w,5,Wow - this place is truly amazing! We stopped...,6,cMEtAiW60I5wE_vLfTxoJQ
3772998,J5ebC7fKqrSF8KuS8d6hgw,3,2016-11-02,1,RSesmosvbJ-LpRGQpMKR_Q,5,I forgot just how amazing this place is! I ha...,2,cMEtAiW60I5wE_vLfTxoJQ
3730129,0CElIQnJdQYF2bmsAiWczg,3,2016-07-01,1,Pjxm973pSy5MZfRb8y6X7A,5,We were anxious to check out the Orchard since...,3,cMEtAiW60I5wE_vLfTxoJQ
3700469,UNfiku8qEyvGYjPK7w_gxQ,2,2013-08-26,0,IxpxcWuAG-tJkn2dv5tncQ,5,I can't believe such amazing food comes out of...,0,cMEtAiW60I5wE_vLfTxoJQ


In [143]:
user1_df = pd.merge(user1_reviews_df,clusterdf, on="business_id")

In [144]:
user1_df

,business_id,cool,date,funny,review_id,stars_x,text,useful,user_id,name,city,categories,RestaurantsPriceRange,stars_y,cluster
0,lCd20nDS9rFHW_2KViqWUA,1,2015-07-04,1,SoF7qlkOh80Z9xblx6MDfw,5,This place is ridiculously good...so good I ca...,4,cMEtAiW60I5wE_vLfTxoJQ,El Chullo Peruvian Restaurant & Bar,Phoenix,"Peruvian,Restaurants",2,4.5,4
1,QARy4UUOQogMjDp9jP-YVg,0,2013-01-30,0,MPFZt68Xgoi5vXY45fYs7w,5,I was really impressed with this place! The d...,0,cMEtAiW60I5wE_vLfTxoJQ,Searsucker,Scottsdale,"Restaurants,American (New)",3,3.5,2
2,Tw3miGKZHtmxmaQZIYFRrA,0,2012-11-03,0,nmrXaPFSUpIlXSWERYYTpw,5,Great new addition to the Central/Camelback co...,0,cMEtAiW60I5wE_vLfTxoJQ,Federal Pizza,Phoenix,"Restaurants,Italian,Pizza",2,4.5,0
3,oj8qk3W7O9oq15VIs-vYJQ,2,2011-05-06,0,C5t4yBkSs6uy-_2AmqYEMQ,5,One of my favorite places in Scottsdale. This...,1,cMEtAiW60I5wE_vLfTxoJQ,Lon's At the Hermosa,Paradise Valley,"Bars,Nightlife,American (Traditional),Restaurants",3,4.0,3
4,Rg43a31sV7gNLZMjy-ZdCg,0,2013-11-17,0,yH1mM3iVGotl4iqUOLNDZw,5,Such a great place! We stopped by this aftern...,2,cMEtAiW60I5wE_vLfTxoJQ,Pig & Pickle,Scottsdale,"Gastropubs,Sandwiches,Restaurants,Comfort Food",2,4.0,0
5,Noi53T0PWNEN9mQRS3-Ncg,0,2015-11-28,0,K-xKU62rckdg3YbPZqsyMg,5,"I had always heard this place was amazing, but...",2,cMEtAiW60I5wE_vLfTxoJQ,D'Lite Healthy On The Go,Scottsdale,"Coffee & Tea,Restaurants,Sandwiches,Food,Break...",1,4.5,0
6,WB7346WSdkTHvtHfWPilaA,5,2015-12-06,5,QszUgcqQ0lNb-5ytFnHW6w,5,Wow - this place is truly amazing! We stopped...,6,cMEtAiW60I5wE_vLfTxoJQ,Be Coffee + Food + Stuff,Phoenix,"Breakfast & Brunch,Food,Coffee & Tea,Restauran...",1,4.5,4
7,J5ebC7fKqrSF8KuS8d6hgw,3,2016-11-02,1,RSesmosvbJ-LpRGQpMKR_Q,5,I forgot just how amazing this place is! I ha...,2,cMEtAiW60I5wE_vLfTxoJQ,Sierra Bonita Grill,Phoenix,"Restaurants,American (Traditional),Burgers,Bar...",2,4.0,3
8,0CElIQnJdQYF2bmsAiWczg,3,2016-07-01,1,Pjxm973pSy5MZfRb8y6X7A,5,We were anxious to check out the Orchard since...,3,cMEtAiW60I5wE_vLfTxoJQ,Luci's at the Orchard,Phoenix,"Restaurants,Event Planning & Services,Breakfas...",2,4.0,0
9,UNfiku8qEyvGYjPK7w_gxQ,2,2013-08-26,0,IxpxcWuAG-tJkn2dv5tncQ,5,I can't believe such amazing food comes out of...,0,cMEtAiW60I5wE_vLfTxoJQ,Gertrude's,Phoenix,"Breakfast & Brunch,Restaurants,Coffee & Tea,Fo...",2,3.5,1


In [145]:
user1_df_cluster1 = user1_df.loc[user1_df['cluster'] == 4]
user1_df_cluster1

,business_id,cool,date,funny,review_id,stars_x,text,useful,user_id,name,city,categories,RestaurantsPriceRange,stars_y,cluster
0,lCd20nDS9rFHW_2KViqWUA,1,2015-07-04,1,SoF7qlkOh80Z9xblx6MDfw,5,This place is ridiculously good...so good I ca...,4,cMEtAiW60I5wE_vLfTxoJQ,El Chullo Peruvian Restaurant & Bar,Phoenix,"Peruvian,Restaurants",2,4.5,4
6,WB7346WSdkTHvtHfWPilaA,5,2015-12-06,5,QszUgcqQ0lNb-5ytFnHW6w,5,Wow - this place is truly amazing! We stopped...,6,cMEtAiW60I5wE_vLfTxoJQ,Be Coffee + Food + Stuff,Phoenix,"Breakfast & Brunch,Food,Coffee & Tea,Restauran...",1,4.5,4
36,DaVTuhzi6EgWStb2eAjNjA,3,2016-02-28,1,2G4LGaM5zNXit1nYiqHvGQ,5,We stopped in today for lunch after reading th...,4,cMEtAiW60I5wE_vLfTxoJQ,Presidio Cocina Mexicana,Phoenix,"Mexican,Nightlife,Food,Cafes,Beer,Wine & Spiri...",2,4.5,4
41,xZtKQ6AFLh9P7rfmjvVjhw,0,2010-01-29,0,ttF2m8Ic_qs4VohC5jUb0g,5,This place is surprisingly good...the fry brea...,0,cMEtAiW60I5wE_vLfTxoJQ,Fry Bread House,Phoenix,"Restaurants,Mexican,Indian,American (Tradition...",1,4.0,4
42,vsFFbN71ehRCp46KeR5RdQ,1,2015-09-02,1,0H-PttnnTbHaYaBQXH3SeQ,5,Great breakfast...definitely worth the wait! ...,4,cMEtAiW60I5wE_vLfTxoJQ,Butterfield's Pancake House,Scottsdale,"Sandwiches,Breakfast & Brunch,American (Tradit...",2,4.5,4
55,frCxZS7lPhEnQRJ3UY6m7A,1,2011-04-25,0,xLLE-u8B8EO8Nwhq76oTng,5,Wow...this place is so good it is ridiculous! ...,2,cMEtAiW60I5wE_vLfTxoJQ,La Santisima,Phoenix,"Bars,Mexican,Vegan,Breakfast & Brunch,Cafes,Ve...",2,4.5,4
63,Grr2wJ3nYD_7qUyespDglA,1,2016-05-09,1,GFS2FLXdnNA0l3xeGP37yQ,5,I arrived in Toronto last Tuesday evening and ...,2,cMEtAiW60I5wE_vLfTxoJQ,Corrado's,Toronto,"Restaurants,Greek,Italian,Pizza",1,4.5,4
73,yHBhLawYmIthUPLbUXK5kw,0,2011-09-22,0,DOjpNdD8pJk-_NidcMzdvw,5,I love this place! Make sure Matt is your bar...,1,cMEtAiW60I5wE_vLfTxoJQ,Fuego Tacos,Phoenix,"Latin American,Restaurants",2,3.5,4
81,V2oe1HVgNK0He9Mf3Q17AA,3,2016-01-25,2,eLgQ8RSXHFOYdyihwJZnWg,5,Wow...how did I not know that this place exist...,6,cMEtAiW60I5wE_vLfTxoJQ,All Pierogi Euro Kitchen & Euro Market,Mesa,"Polish,Ethnic Food,Russian,Specialty Food,Groc...",2,4.5,4
85,L2p0vO3fsS2LC6hhQo3CzA,4,2016-11-29,1,AhflsEIt9OCm_19H4PZT0w,5,Wow...I can't believe I haven't been here in y...,3,cMEtAiW60I5wE_vLfTxoJQ,Matt's Big Breakfast,Phoenix,"Breakfast & Brunch,Restaurants",2,4.0,4


In [124]:
Counter(user1_df_cluster1['stars_x'].tolist())

Counter({1: 1, 2: 3, 3: 39, 4: 40, 5: 7})

In [131]:
Counter(user1_df_cluster1['stars_x'].tolist())

Counter({1: 1, 2: 5, 3: 34, 4: 43, 5: 17})

In [139]:
Counter(user1_df_cluster1['stars_x'].tolist())

Counter({1: 3, 2: 5, 3: 62, 4: 70, 5: 14})

In [146]:
Counter(user1_df_cluster1['stars_x'].tolist())

Counter({2: 1, 3: 10, 4: 29, 5: 12})

# Normalize featurematrix

In [68]:
resdf = pd.read_csv("business_final.csv")

In [87]:
# retreive phonex and las vegas restaurants
restaurants = pd.read_csv('rest.csv')

In [88]:
restaurants

,Id,Name,City,count,Words
0,sNVGdeOPeitJ3OWUQBINzQ,BabyStacks Cafe,Las Vegas,1,"['recommend', 'amaz', 'vega', 'best', 'ever', ..."
1,vl2IZrNJEA8npSjqXbdwxw,BabyStacks Cafe,Las Vegas,1,"['worth', 'best', 'amaz', 'delici', 'alway', '..."
2,RESDUcs7fIiihp38-d6_6g,Bacchanal Buffet,Las Vegas,1,"['everi', 'worth', 'ever', 'favorit', 'best', ..."
3,7sPNbCx7vGAaH7SbNPZ6oA,Bachi Burger,Las Vegas,1,"['best', 'amaz', 'ever', 'vega', '5', 'perfect..."
4,ZibmYdOPKLlqDM9oR6xzOA,Bachi Burger,Las Vegas,1,"['best', 'vega', 'amaz', 'ever', 'awesom', 'ti..."
5,YhCAJ8acd1X7GkCHPhD8Xw,BLT Burger,Las Vegas,1,"['best', 'vega', 'ever', 'awesom', 'must', 'am..."
6,d_L-rfS1vT3JMzgCUGtiow,Border Grill,Las Vegas,1,"['amaz', 'best', 'us', 'made', 'perfect', 'bru..."
7,rcaPajgKOJC2vo_l3xa42A,Bouchon at the Venezia Tower,Las Vegas,1,"['amaz', 'perfect', 'best', 'ever', 'favorit',..."
8,aT_SsfZ6GQgJGyuIv1Hapw,Bouchon Bakery at the Venetian Casino,Las Vegas,1,"['ever', 'love', 'worth', 'everi', 'also', 'pe..."
9,IhNASEZ3XnBHmuuVnWdIwA,Brew Tea Bar,Las Vegas,1,"['alway', 'best', 'great', 'favorit', 'amaz', ..."


In [89]:
allRest = restaurants['Id'].tolist()

In [90]:
allRest

['sNVGdeOPeitJ3OWUQBINzQ',
 'vl2IZrNJEA8npSjqXbdwxw',
 'RESDUcs7fIiihp38-d6_6g',
 '7sPNbCx7vGAaH7SbNPZ6oA',
 'ZibmYdOPKLlqDM9oR6xzOA',
 'YhCAJ8acd1X7GkCHPhD8Xw',
 'd_L-rfS1vT3JMzgCUGtiow',
 'rcaPajgKOJC2vo_l3xa42A',
 'aT_SsfZ6GQgJGyuIv1Hapw',
 'IhNASEZ3XnBHmuuVnWdIwA',
 'nUpz0YiBsOK7ff9k3vUJ3A',
 'Cni2l-VKG_pdospJ6xliXQ',
 'Jt28TYWanzKrJYYr0Tf1MQ',
 '7fxebHYUwIF6CakxSr70iQ',
 'FLMxWQO-ckCQmGZhU9OQgw',
 '2iTsRqUsPGRH1li1WVRvKQ',
 'oXoVJ0xKv82cBo9U6oEjlQ',
 'GJ_bXUPv672YwNg4TneJog',
 'OVTZNSkSfbl3gVB9XQIJfw',
 'J4CATH00YZrq8Bne2S4_cw',
 'IT_4EEIbv6Ox1jBRMyE7pg',
 '--9e1ONYQuAa-CB_Rrw7Tw',
 '0AQnRQw34IQW9-1gJkYnMA',
 'qqs7LP4TXAoOrSlaKRfz3A',
 'DkYS3arLOhA8si5uUEmHOw',
 'l_GV0hgEoTUf70uJVT0_hg',
 'igHYkXZMLAc9UdV5VnR_AA',
 '3kdSl5mo9dWC4clrQjEDGg',
 'PXShA3JZMXr2mEH3on5clw',
 'xVEtGucSRLk5pxxN0t4i6g',
 'JDZ6_yycNQFTpUZzLIKHUg',
 'EAwh1OmG6t6p3nRaZOW_AA',
 '5shgJB7a-2_gdnzc0gsOtg',
 'kRgAf6j2y1eR0wOFdzFAuw',
 'hihud--QRriCYZw1zZvW4g',
 'G-5kEa6E6PD5fkBRuA7k9Q',
 'cYwJA2A6I12KNkm2rtXd5g',
 

In [69]:
resdf

,business_id,name,city,categories,romantic,intimate,classy,hipster,divey,touristy,...,Chinese,Coffee & Tea,Vegetarian,Desserts,Mediterranean,Thai,Barbeque,Bakeries,French,Indian
0,4JNXUYY8wbaaDmk3BPzlWw,Mon Ami Gabi,Las Vegas,"French,Steakhouses,Breakfast & Brunch,Restaurants",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,RESDUcs7fIiihp38-d6_6g,Bacchanal Buffet,Las Vegas,"Food,Restaurants,Buffets,Breakfast & Brunch,Sa...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,K7lWdNUhCbcnEvI0NhGewg,Wicked Spoon,Las Vegas,"Buffets,Breakfast & Brunch,Restaurants",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,cYwJA2A6I12KNkm2rtXd5g,Gordon Ramsay BurGR,Las Vegas,"American (Traditional),Restaurants,Burgers",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DkYS3arLOhA8si5uUEmHOw,Earl of Sandwich,Las Vegas,"Food Delivery Services,Restaurants,Caterers,Sa...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,f4x1YBxkLrZg652xt2KR5g,Hash House A Go Go,Las Vegas,"American (New),Restaurants,Breakfast & Brunch",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,eoHdUeQDNgQ6WYEnP2aiRw,Serendipity 3,Las Vegas,"Social Clubs,Restaurants,Desserts,Food,America...",0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
7,2weQS-RnoOBhb1KsHKyoSQ,The Buffet,Las Vegas,"Buffets,Restaurants",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,KskYqH1Bi7Z_61pH6Om8pg,Lotus of Siam,Las Vegas,"Automotive,Car Dealers,Restaurants,Thai,Nightl...",0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,ujHiaprwCQ5ewziu0Vi9rw,The Buffet at Bellagio,Las Vegas,"American (New),Buffets,Restaurants",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
all_business = resdf['business_id'].tolist()

In [45]:
featurematrix = resdf.drop(['name','business_id','city','categories'], axis=1)

In [46]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(featurematrix)
df_normalized = pd.DataFrame(np_scaled)
df_normalized

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Kmeans 

In [49]:
from sklearn.cluster import KMeans

# Convert DataFrame to matrix
mat = df_normalized.as_matrix()
# Using sklearn
km = KMeans(n_clusters=5)
km.fit(mat)
# Get cluster assignment labels
labels = km.labels_
# Format results as a DataFrame
res_labels = pd.DataFrame([labels]).T
res_labels.columns = ['cluster']

In [55]:
res_labels

,cluster
0,1
1,4
2,1
3,2
4,4
5,4
6,2
7,1
8,4
9,4


In [56]:
clusterdf = resdf.join(res_labels)

In [93]:
selected_df = resdf.filter(['business_id','name','city','categories','RestaurantsPriceRange','stars'], axis=1)

In [94]:
clusterdf = selected_df.join(res_labels)

In [95]:
clusterdf

,business_id,name,city,categories,RestaurantsPriceRange,stars,cluster
0,4JNXUYY8wbaaDmk3BPzlWw,Mon Ami Gabi,Las Vegas,"French,Steakhouses,Breakfast & Brunch,Restaurants",2,4.0,1
1,RESDUcs7fIiihp38-d6_6g,Bacchanal Buffet,Las Vegas,"Food,Restaurants,Buffets,Breakfast & Brunch,Sa...",3,4.0,4
2,K7lWdNUhCbcnEvI0NhGewg,Wicked Spoon,Las Vegas,"Buffets,Breakfast & Brunch,Restaurants",3,3.5,1
3,cYwJA2A6I12KNkm2rtXd5g,Gordon Ramsay BurGR,Las Vegas,"American (Traditional),Restaurants,Burgers",2,4.0,2
4,DkYS3arLOhA8si5uUEmHOw,Earl of Sandwich,Las Vegas,"Food Delivery Services,Restaurants,Caterers,Sa...",1,4.5,4
5,f4x1YBxkLrZg652xt2KR5g,Hash House A Go Go,Las Vegas,"American (New),Restaurants,Breakfast & Brunch",2,4.0,4
6,eoHdUeQDNgQ6WYEnP2aiRw,Serendipity 3,Las Vegas,"Social Clubs,Restaurants,Desserts,Food,America...",2,3.0,2
7,2weQS-RnoOBhb1KsHKyoSQ,The Buffet,Las Vegas,"Buffets,Restaurants",3,3.5,1
8,KskYqH1Bi7Z_61pH6Om8pg,Lotus of Siam,Las Vegas,"Automotive,Car Dealers,Restaurants,Thai,Nightl...",2,4.0,4
9,ujHiaprwCQ5ewziu0Vi9rw,The Buffet at Bellagio,Las Vegas,"American (New),Buffets,Restaurants",3,3.5,4


In [98]:
# for only las vegas and phoneix
new_cluster = clusterdf[clusterdf.business_id.apply(lambda x: x in allRest)]

In [99]:
new_cluster.to_csv('business_cluster_2.csv', index=False)

In [80]:
# for all restaurants
clusterdf.to_csv('business_cluster.csv', index=False)

In [74]:
cluser_1 = clusterdf.loc[clusterdf['cluster'] == 1]

In [75]:
cluser_1

,business_id,name,city,categories,cluster
0,4JNXUYY8wbaaDmk3BPzlWw,Mon Ami Gabi,Las Vegas,"French,Steakhouses,Breakfast & Brunch,Restaurants",1
2,K7lWdNUhCbcnEvI0NhGewg,Wicked Spoon,Las Vegas,"Buffets,Breakfast & Brunch,Restaurants",1
7,2weQS-RnoOBhb1KsHKyoSQ,The Buffet,Las Vegas,"Buffets,Restaurants",1
13,hihud--QRriCYZw1zZvW4g,Gangnam Asian BBQ Dining,Las Vegas,"Restaurants,Barbeque,Asian Fusion,Japanese,Tap...",1
14,g8OnV26ywJlZpezdBnOWUQ,Hash House A Go Go,Las Vegas,"American (New),Restaurants,Breakfast & Brunch",1
19,XXW_OFaYQkkGOGniujZFHg,The Peppermill Restaurant & Fireside Lounge,Las Vegas,"Nightlife,Lounges,Restaurants,Bars,American (T...",1
31,ECOkEVUodMLUxvI0PMI4gQ,Tao Nightclub,Las Vegas,"Arts & Entertainment,Restaurants,Lounges,Bars,...",1
35,pH0BLkL4cbxKzu471VZnuA,SUSHISAMBA - Las Vegas,Las Vegas,"Nightlife,Restaurants,Breakfast & Brunch,Asian...",1
44,0W4lkclzZThpx3V65bVgig,Schwartz's,Montr̩al,"Restaurants,Delis,Specialty Food,Imported Food...",1
46,VyVIneSU7XAWgMBllI6LnQ,Bobby Q,Phoenix,"Restaurants,Barbeque,Event Planning & Services...",1


## Find all the top cuisines

In [33]:
cuisine = set()
all_cuisine = []
for index, row in df.iterrows():
    k = row['categories'].split(',')
    for i in k:
        cuisine.add(i)
        all_cuisine.append(i)

In [36]:
from collections import Counter
cuisine_counts = Counter(all_cuisine)

In [32]:
print len(cuisine)

290


In [37]:
print cuisine_counts

Counter({'Restaurants': 5000, 'Nightlife': 1181, 'Bars': 1149, 'Food': 1034, 'American (New)': 914, 'American (Traditional)': 808, 'Breakfast & Brunch': 725, 'Italian': 514, 'Mexican': 489, 'Pizza': 465, 'Sandwiches': 457, 'Seafood': 413, 'Japanese': 395, 'Burgers': 387, 'Sushi Bars': 362, 'Asian Fusion': 302, 'Steakhouses': 300, 'Chinese': 270, 'Salad': 254, 'Cafes': 230, 'Coffee & Tea': 213, 'Vegetarian': 212, 'Desserts': 207, 'Event Planning & Services': 205, 'Mediterranean': 201, 'Thai': 191, 'Barbeque': 190, 'Sports Bars': 183, 'Wine Bars': 181, 'Cocktail Bars': 166, 'Pubs': 161, 'Beer': 149, 'Wine & Spirits': 149, 'Bakeries': 140, 'Vegan': 137, 'Diners': 135, 'French': 130, 'Arts & Entertainment': 126, 'Gluten-Free': 125, 'Buffets': 123, 'Specialty Food': 121, 'Lounges': 120, 'Fast Food': 119, 'Gastropubs': 118, 'Vietnamese': 115, 'Caterers': 110, 'Greek': 106, 'Chicken Wings': 103, 'Korean': 99, 'Indian': 94, 'Southern': 89, 'Canadian (New)': 87, 'Tapas/Small Plates': 85, 'Brewe